In [33]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import json

import pandas as pd
pd.set_option("display.max_columns", 30)

import requests

"""
1. get the data from S3
2. weather data transformations
3. taxi trips transformations - DONE
4. update payment_type_master - DONE
5. update company_type_master - DONE
6. update taxi trips with company and payment type ids (replace the string values with ids from the latest master tables) - DONE
7. upload weather data to s3
8. upload taxi data to s3
9. upload the newest payment_type_master and company_master

"""

### taxi_trips transformation codes

In [34]:
# Calculate the date two months ago
current_datetime = datetime.now() - relativedelta(months=2)
formatted_date = current_datetime.strftime("%Y-%m-%d")

# Construct the query URL
url = (
    "https://data.cityofchicago.org/resource/ajtu-isnz.json?"
    f"$where=trip_start_timestamp >= '{formatted_date}T00:00:00'"
    f" AND trip_start_timestamp <= '{formatted_date}T23:59:59'"
    f"&$limit=30000"
)

# Make the GET request
response = requests.get(url)

data = response.json()

In [35]:
taxi_trips = pd.DataFrame(data)

In [36]:
taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract"], axis=1, inplace=True)
taxi_trips.drop(["pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True)

taxi_trips.dropna(inplace=True)

taxi_trips.rename(columns={"pickup_community_area": "pickup_community_area_id",
                           "dropoff_community_area": "dropoff_community_area_id"}, inplace=True)

taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_start_timestamp"]).dt.floor("h")

In [37]:
taxi_trips.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
0,9caf1197a8eaffa9468b9ff1eb7669a958bafad9,1a248eb87ae578f15af2a14c2eb7b4b264f6babab9ad9c...,2024-10-03T23:45:00.000,2024-10-04T00:00:00.000,626,3.52,8,7,12,3.12,0,0,15.62,Credit Card,Sun Taxi,41.899602111,-87.633308037,41.922686284,-87.649488729,2024-10-03 23:00:00
1,fb421a6d1a45dc15311ba10f30f448e583718bdb,48d462bcf24c2aadcda3fcf689d3f63fc178f23dc73bba...,2024-10-03T23:45:00.000,2024-10-04T00:00:00.000,835,4.48,8,33,12.73,3.32,0,0,16.05,Mobile,Sun Taxi,41.899602111,-87.633308037,41.857183858,-87.620334624,2024-10-03 23:00:00
2,f67243ee732e06bb4e2d6b0dd7cc98d39951c016,e1cf6c401b0eff9128dad27435a344db33ed69d2be0a63...,2024-10-03T23:45:00.000,2024-10-04T00:00:00.000,1088,5.86,6,28,15.37,3.77,0,0,19.14,Mobile,5 Star Taxi,41.944226601,-87.655998182,41.874005383,-87.66351755,2024-10-03 23:00:00
3,f292fd839b7f5e901124c899421586b59551fe31,b77a2dcc078698ea493d4d703014076e4272dc7d8b420e...,2024-10-03T23:45:00.000,2024-10-04T00:15:00.000,1860,22.7,76,61,54.25,11.95,0,5,71.2,Credit Card,Taxi Affiliation Services,41.980264315,-87.913624596,41.80901825,-87.659166599,2024-10-03 23:00:00
4,ecc4413872b0dfc42f14359dd721bff99734570a,30b72bb886e483d8dfd1adc7eff86789a677e3732ccd5c...,2024-10-03T23:45:00.000,2024-10-04T00:15:00.000,1469,17.92,76,8,44.5,10,0,5,60,Credit Card,City Service,41.980264315,-87.913624596,41.899602111,-87.633308037,2024-10-03 23:00:00


### taxi trips transformation function

In [38]:
def taxi_trips_transformations(taxi_trips: pd.DataFrame) -> pd.DataFrame:
    """
    Perform transformations with the taxi data

    Parameters:
        taxi_trips: pd.DataFrame
            The DataFrame holding the daily taxi trips

    Returns:
        pd.DataFrame
            The cleaned, transformed DataFrame holding the daily taxi trips.
    """

    if not isinstance(taxi_trips, pd.DataFrame):
        raise TypeError("taxi_trips is not a valid pandas DataFrame.")

    taxi_trips.drop(
        ["pickup_census_tract", "dropoff_census_tract",
         "pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True)

    taxi_trips.dropna(inplace=True)

    taxi_trips.rename(columns={"pickup_community_area": "pickup_community_area_id",
                               "dropoff_community_area": "dropoff_community_area_id"}, inplace=True)

    taxi_trips["datetime_for_weather"] = pd.to_datetime(taxi_trips["trip_start_timestamp"]).dt.floor("h")

    return taxi_trips

### company code updates

In [39]:
company_master = taxi_trips["company"].drop_duplicates().reset_index(drop=True)

company_master = pd.DataFrame({
    "company_id": range(1, len(company_master) + 1),

    "company": company_master
})

company_master.tail()

,company_id,company
28,29,5167 - 71969 5167 Taxi Inc
29,30,3556 - 36214 RC Andrews Cab
30,31,Tac - Yellow Non Color
31,32,Metro Jet Taxi A.
32,33,4053 - 40193 Adwar H. Nikola


In [40]:
# new_company_data = [
#     {"company": "6574 - Babylon Express Inc."},
#     {"company": "X"},
#     {"company": "Y"}
# ]

# new_company_mapping = pd.DataFrame(new_company_data)

# new_company_mapping

In [41]:
# company_max_id = company_master["company_id"].max()

# company_max_id

In [42]:
# new_company_mapping["company_id"] = range(company_max_id + 1, company_max_id + len(new_company_mapping) + 1)

# new_company_mapping


In [43]:
# new_companies = new_company_mapping[~new_company_mapping["company"].isin(company_master["company"])]

# new_companies

In [44]:
# if not new_companies.empty:
#     company_master = pd.concat([company_master, new_companies], ignore_index=True)

# company_master.tail()

##### new approach to do the same

In [45]:
company_max_id = company_master["company_id"].max()
company_max_id

np.int64(33)

In [46]:
new_companies_list = []

for company in new_company_mapping["company"].values:
    if company not in company_master["company"].values:
        new_companies_list.append(company)

# one line
new_companies_list = [company for company in new_company_mapping["company"].values if company not in company_master["company"].values]

new_companies_list

['X', 'Y']

In [47]:
new_companies_df = pd.DataFrame({
    "company_id": range(company_max_id + 1, company_max_id + len(new_companies_list) + 1),
    "company": new_companies_list
})

new_companies_df

,company_id,company
0,34,X
1,35,Y


In [48]:
updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)

updated_company_master.tail()

,company_id,company
30,31,Tac - Yellow Non Color
31,32,Metro Jet Taxi A.
32,33,4053 - 40193 Adwar H. Nikola
33,34,X
34,35,Y


In [49]:
def update_company_master(taxi_trips: pd.DataFrame, company_master: pd.DataFrame) -> pd.DataFrame:
    """
    Extend the company master with new companies if there are new companies.

    Parameters
    ----------
    taxi_trips: pd.DataFrame
        DataFrame holding the daily taxi trips.
    company_master: pd.DataFrame
        DataFrame holding the company master data.

    Returns
    -------
    pd.DataFrame
        The updated company_master data, if new companies are in the taxi data, they will be loaded to it.
    """

    company_max_id = company_master["company_id"].max()
    new_companies_list = [company for company in taxi_trips["company"].values if company not in company_master["company"].values]

    new_companies_df = pd.DataFrame({
        "company_id": range(company_max_id + 1, company_max_id + len(new_companies_list) + 1),
        "company": new_companies_list
    })

    updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)

    return updated_company_master

In [50]:
test_df = update_company_master(taxi_trips=taxi_trips, company_master=company_master)

In [51]:
test_df

,company_id,company
0,1,Sun Taxi
1,2,5 Star Taxi
2,3,Taxi Affiliation Services
3,4,City Service
4,5,Taxicab Insurance Agency Llc
5,6,Flash Cab
6,7,Tac - American United Dispatch
7,8,Blue Ribbon Taxi Association
8,9,Chicago Independents
9,10,Medallion Leasin


In [52]:
taxi_trips_company_only = pd.DataFrame({
    "company_id": [1, 2, 3],
    "company": ["6574 - Babylon Express Inc.", "X", "Y"]
})

taxi_trips_company_only

,company_id,company
0,1,6574 - Babylon Express Inc.
1,2,X
2,3,Y


In [53]:
updated_company_master = update_company_master(taxi_trips=taxi_trips_company_only, company_master=company_master)

In [54]:
updated_company_master.tail()

,company_id,company
30,31,Tac - Yellow Non Color
31,32,Metro Jet Taxi A.
32,33,4053 - 40193 Adwar H. Nikola
33,34,X
34,35,Y


### payment_type_master_codes

In [58]:
payment_type_master = taxi_trips["payment_type"].drop_duplicates().reset_index(drop=True)

payment_type_master = pd.DataFrame({
    "payment_type_id": range(1, len(payment_type_master) + 1),
    "payment_type": payment_type_master
})

taxi_trips_payment_type_only = pd.DataFrame({
    "payment_type_id": [1, 2, 3],
    "payment_type": ["Credit Card", "X", "Y"]
})


taxi_trips_payment_type_only

,payment_type_id,payment_type
0,1,Credit Card
1,2,X
2,3,Y


In [59]:
def update_payment_type_master(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame) -> pd.DataFrame:
    """
    Extend the payment type master with new payment types if there are new payment types.

    Parameters
    ----------
    taxi_trips: pd.DataFrame
        DataFrame holding the daily taxi trips.
    payment_type_master: pd.DataFrame
        DataFrame holding the payment type master data.

    Returns
    -------
    pd.DataFrame
        The updated payment type master data, if new payment types are in the taxi data, they will be loaded to it.
    """

    payment_type_max_id = payment_type_master["payment_type_id"].max()
    new_payment_types_list = [payment_type for payment_type in taxi_trips["payment_type"].values if payment_type not in payment_type_master["payment_type"].values]
    new_payment_type_df = pd.DataFrame({
        "payment_type_id": range(payment_type_max_id + 1, payment_type_max_id + len(new_payment_types_list) + 1),
        "payment_type": new_payment_types_list
    })

    updated_payment_type_master = pd.concat([payment_type_master, new_payment_type_df], ignore_index=True)

    return updated_payment_type_master

In [60]:
updated_payment_type_master = update_payment_type_master(taxi_trips=taxi_trips_payment_type_only, payment_type_master=payment_type_master)

In [61]:
updated_payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Mobile
2,3,Cash
3,4,Prcard
4,5,Unknown
5,6,Dispute
6,7,No Charge
7,8,X
8,9,Y


### Creating a generic update master table function

In [ ]:
def update_master(taxi_trips: pd.DataFrame, master: pd.DataFrame, id_column: str, value_column: str) -> pd.DataFrame:
    """
    Extend the master DataFrame with new values if there are any.

    Parameters
    ----------
    taxi_trips: pd.DataFrame
        DataFrame holding the daily taxi trips.
    master: pd.DataFrame
        DataFrame holding the master data.
    id_column: str
        The id column of the master DataFrame.
    value_column: str
        Name of the column in master_df containing the values.

    Returns
    -------
    pd.DataFrame
        The updated master data, if new values are in the taxi data, they will be loaded to it.
    """

    max_id = master[id_column].max()

    new_values_list = list(set(taxi_trips[value_column].values) - set(master[value_column].values))

    new_values_df = pd.DataFrame({
        id_column: range(max_id + 1, max_id + len(new_values_list) + 1),
        value_column: new_values_list
    })

    updated_master = pd.concat([master, new_values_df], ignore_index=True)

    return updated_master

In [63]:
test_payment_type_master = update_master(taxi_trips=taxi_trips_payment_type_only, master=payment_type_master, id_column="payment_type_id", value_column="payment_type")

In [64]:
test_payment_type_master = update_master(taxi_trips=taxi_trips_payment_type_only, master=payment_type_master, id_column="payment_type_id", value_column="payment_type")

In [65]:
test_payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Mobile
2,3,Cash
3,4,Prcard
4,5,Unknown
5,6,Dispute
6,7,No Charge
7,8,X
8,9,Y


In [66]:
test_company_master = update_master(taxi_trips=taxi_trips_company_only, master=company_master, id_column="company_id", value_column="company")

In [67]:
test_company_master.tail()

,company_id,company
30,31,Tac - Yellow Non Color
31,32,Metro Jet Taxi A.
32,33,4053 - 40193 Adwar H. Nikola
33,34,X
34,35,Y


### Update taxi_trips with the most recent company_master and payment_type_master codes

In [69]:
taxi_trips_id = taxi_trips.merge(payment_type_master, on="payment_type")
taxi_trips_id = taxi_trips_id.merge(company_master, on="company")

In [71]:
taxi_trips_id.sample(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,payment_type_id,company_id
1912,7548b2dbaf4c09be24ac9e4851d95fbab29e88fc,b41e8c8124d8d600a4db8b7c176c2956756d378c749cb1...,2024-10-03T20:15:00.000,2024-10-03T20:30:00.000,600,0,8,8,14.75,0,0,0,14.75,No Charge,Taxi Affiliation Services,41.899155613,-87.626210532,41.890922026,-87.618868355,2024-10-03 20:00:00,7,3
7541,6abce90d952de33ab3e44b2540cc68edaeaf879d,f2721c7874859e2db0826b73aaa9dfe1ff67c606a9fd75...,2024-10-03T15:15:00.000,2024-10-03T15:30:00.000,840,3.4,8,7,12.5,0,0,0,12.5,Cash,Taxi Affiliation Services,41.899602111,-87.633308037,41.922686284,-87.649488729,2024-10-03 15:00:00,3,3
2129,5db6bdd6993d4d05db2daed7cf420a7ce02f1ec5,179f1a051e9e6d3fc0726628962faff68506086ee8df14...,2024-10-03T20:00:00.000,2024-10-03T20:30:00.000,1440,10.33,76,14,27.25,4.76,0,4,36.51,Credit Card,Blue Ribbon Taxi Association,41.980264315,-87.913624596,41.968069,-87.721559063,2024-10-03 20:00:00,1,8
4358,b26cf112393c5f8c1033d4bb166749420745b828,0e8e342e2e2db3801515e84538a3901dbea9df9cd1716b...,2024-10-03T17:45:00.000,2024-10-03T18:00:00.000,120,0.3,32,28,4.25,1,0,0,5.25,Credit Card,Taxi Affiliation Services,41.880994471,-87.632746489,41.885300022,-87.642808466,2024-10-03 17:00:00,1,3
5107,364cd7f22bfe31fb8d72b9385b533bd7cf5720e8,7460d7f323c633150b08d9fb110001eb3189c31e74a915...,2024-10-03T17:15:00.000,2024-10-03T17:30:00.000,480,1.4,8,28,7.5,0,0,0,7.5,Cash,Taxi Affiliation Services,41.892072635,-87.628874157,41.879255084,-87.642648998,2024-10-03 17:00:00,3,3


In [72]:
def update_taxi_trips_with_master_data(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame, company_master: pd.DataFrame) -> pd.DataFrame:
    """
    Update the taxi_trips DataFrame with the company master and payment type master ids, and delete the string columns.

    Parameters
    ----------
    taxi_trips: pd.DataFrame
        The DataFrame with the daily taxi trips.
    payment_type_master: pd.DataFrame
        The payment type master table.
    company_master: pd.DataFrame
        The company master table.

    Returns
    -------
    pd.DataFrame
        The taxi trips data, with only payment_type_id and company_id, without company or payment_type values.
    """

    taxi_trips_id = taxi_trips.merge(payment_type_master, on="payment_type")
    taxi_trips_id = taxi_trips_id.merge(company_master, on="company")

    taxi_trips_id.drop(["payment_type", "company"], axis=1, inplace=True)

    return taxi_trips_id

In [73]:
taxi_trips_id = update_taxi_trips_with_master_data(taxi_trips=taxi_trips, payment_type_master=payment_type_master, company_master=company_master)

taxi_trips_id.sample(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,payment_type_id,company_id
16848,8863d3ac2d6c67ddef6a1c1d0e523376ed9bc357,bff345061f21d319fb6ec44e0ab2bb05478383a21e1ecb...,2024-10-03T05:00:00.000,2024-10-03T05:15:00.000,1197,11.8,77,33,30.75,0,0,0,30.75,41.9867118,-87.663416405,41.857183858,-87.620334624,2024-10-03 05:00:00,4,6
20,c1e6bc5788448bf3f8f697533a3f8a8807bf120c,ae708f295169b3a30fb94aaaf518cd93baae96390a5108...,2024-10-03T23:45:00.000,2024-10-04T00:15:00.000,1935,0,56,56,14,0,0,5,19,41.79259236,-87.769615453,41.79259236,-87.769615453,2024-10-03 23:00:00,3,8
16434,811782afd6eba89990cb00b11ecdbfc8d95433b0,d6111a4915fc052d5390e9002ffa1cd90f667fd20b8efc...,2024-10-03T06:45:00.000,2024-10-03T07:00:00.000,395,1.57,28,32,7.25,0,0,0,7.25,41.874005383,-87.66351755,41.878865584,-87.625192142,2024-10-03 06:00:00,3,8
13547,daa36bff547532937deb541c9b8d98b527fa71d2,0ec47f4701f91d7ec5be91fc7b7e4a05e3bf591444ca4f...,2024-10-03T09:45:00.000,2024-10-03T10:00:00.000,606,4.47,76,76,13.5,3.6,0,4,21.6,41.980264315,-87.913624596,41.980264315,-87.913624596,2024-10-03 09:00:00,1,4
585,38f64d48b463c94a4b55d1641653c2963d12b4e1,835869d6df51f9ec6e53cb4ac6a1f9e386c26f948cdcca...,2024-10-03T22:15:00.000,2024-10-03T22:45:00.000,1395,16.81,76,28,41.25,0,0,4,45.75,41.980264315,-87.913624596,41.874005383,-87.66351755,2024-10-03 22:00:00,1,5


### weather transformations function

In [74]:
def transform_weather_data(weather_data: json) -> pd.DataFrame:
    """
    Make transformations on the daily weather api response.

    Parameters
    ----------
    weather_data: json
        The daily weather data from the Open Meteo API.

    Returns
    -------
    pd.DataFrame
        A DataFrame representation of the data.
    """

    weather_data_filtered = {
        "datetime": weather_data["hourly"]["time"],
        "tempretaure": weather_data["hourly"]["temperature_2m"],
        "wind_speed": weather_data["hourly"]["wind_speed_10m"],
        "rain": weather_data["hourly"]["rain"],
        "precipitation": weather_data["hourly"]["precipitation"],
    }

    weather_df = pd.DataFrame(weather_data_filtered)

    weather_df["datetime"] = pd.to_datetime(weather_df["datetime"])

    return weather_df

In [76]:
# Test

current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d")

url = "https://archive-api.open-meteo.com/v1/era5"

params = {
    "latitude": 41.85,
    "longitude": -87.65,
    "start_date": formatted_datetime,
    "end_date": formatted_datetime,
    "hourly": "temperature_2m,wind_speed_10m,rain,precipitation"
}

response = requests.get(url, params=params)

weather_data = response.json()

weather_data_df = transform_weather_data(weather_data)

In [77]:
weather_data_df.head()

,datetime,tempretaure,wind_speed,rain,precipitation
0,2024-10-03 00:00:00,16.3,14.7,0.0,0.0
1,2024-10-03 01:00:00,16.2,15.7,0.0,0.0
2,2024-10-03 02:00:00,15.4,16.1,0.0,0.0
3,2024-10-03 03:00:00,14.7,15.4,0.0,0.0
4,2024-10-03 04:00:00,14.2,15.8,0.0,0.0


In [78]:
print(range(4, 7))

range(4, 7)


In [79]:
range(4, 7)

range(4, 7)